In [1]:
!apt install tesseract-ocr
!pip install pytesseract
!apt install tesseract-ocr-fra

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,740 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

BILAN ARRETE


Division de l'image sur 2

In [2]:
from PIL import Image

# Charger l'image
image_path = "image007.png"
image = Image.open(image_path)

# Diviser l'image en deux parties verticalement
width, height = image.size
left_part = image.crop((0, 0, width // 2, height))
right_part = image.crop((width // 2, 0, width, height))

# Enregistrer les deux parties
left_part_path = "image007_left.png"
right_part_path = "image007_right.png"
left_part.save(left_part_path)
right_part.save(right_part_path)

print(f"Partie gauche enregistrée à: {left_part_path}")
print(f"Partie droite enregistrée à: {right_part_path}")


Partie gauche enregistrée à: image007_left.png
Partie droite enregistrée à: image007_right.png


Extraction des totaux des actifs

In [3]:
from PIL import Image, ImageEnhance
import pytesseract
import re

# Charger l'image
chemin_image = "image007_left.png"
image = Image.open(chemin_image)

# Convertir l'image en niveaux de gris
gray_image = image.convert('L')

# Augmenter le contraste de l'image
enhancer = ImageEnhance.Contrast(gray_image)
contrasted_image = enhancer.enhance(1.5)  # Augmenter le contraste par un facteur de 2

# Redimensionner l'image (facteur de 1.5)
width, height = contrasted_image.size
resized_image = contrasted_image.resize((int(width * 4.32), int(height * 4.32)), Image.LANCZOS)

# Binariser l'image (seuil ajusté)
binarized_image = resized_image.point(lambda x: 0 if x < 140 else 255, '1')

# Extraire le texte de l'image améliorée avec l'option psm 6
custom_config = r'--oem 3 --psm 6'
texte_bilan_enhanced_actifs = pytesseract.image_to_string(binarized_image, config=custom_config)

# Corriger les mots incorrects
# texte_bilan_enhanced_actifs = re.sub(r'\bTotel\b', 'Total', texte_bilan_enhanced_actifs, flags=re.IGNORECASE)
# texte_bilan_enhanced_actifs = re.sub(r'\bCouraats\b', 'Courants', texte_bilan_enhanced_actifs, flags=re.IGNORECASE)
# texte_bilan_enhanced_actifs = re.sub(r'\bInomobilisés\b', 'Immobilisés', texte_bilan_enhanced_actifs, flags=re.IGNORECASE)

# Fonction pour insérer le séparateur entre les deux valeurs
def insert_separator(line):
    # Expression régulière pour trouver les nombres
    numbers = re.findall(r'\d{1,3}(?:[\s.,]\d{3})*', line)
    if len(numbers) == 2:
        # Remplacer la première occurrence du deuxième nombre par '| <number>'
        line = re.sub(re.escape(numbers[1]), '| ' + numbers[1], line, 1)
    return line


def contains_keywords(text, keywords):
    """Check if the extracted text contains any of the specified keywords."""
    for keyword in keywords:
        if keyword.lower() in text.lower():
            return True
    return False
def extract_financial_data(text, keywords):
    """Extract financial data if the text contains specific keywords."""
    if contains_keywords(text, keywords):
                return "Il s'agit bien d'un bilan comptable:"
    else:
        return "The image does not appear to be a balance sheet."

# Filtrer les lignes pour ne conserver que celles contenant "Total", "total" ou "TOTAL"
lignes = texte_bilan_enhanced_actifs.split('\n')
lignes_filtrees = [insert_separator(ligne) for ligne in lignes if re.search(r'\bTotal\b', ligne, re.IGNORECASE)]

# Rejoindre les lignes filtrées en un seul texte
texte_filtre_actifs = '\n'.join(lignes_filtrees)

keywords = ["Actifs", "Passifs", "Capitaux propres", "Résultat", "ETAT DE RESULTAT"]
extracted_value = extract_financial_data(texte_filtre_actifs, keywords)

texte_filtre_actifs = re.sub(r'\bpo\b', '', texte_filtre_actifs)

# Afficher le texte filtré et s'il 'sagit d'un bilan ou pas
print(extracted_value)
print(texte_filtre_actifs)


Il s'agit bien d'un bilan comptable:
Total des Actifs Immobilisés 10 252 069 | 11 338 516
Total des Actifs non Courants 10 252 069 | 11 338 516
Total des Actifs Courants 47 017 145 | 25 782 633
TOTAL DES ACTIFS  57 269 214 | 37 121 148


Extraction des totaux des passifs et des capitaux propres pour l'image007

In [4]:
from PIL import Image, ImageEnhance
import pytesseract

# Charger l'image
chemin_image = "image007_right.png"
image = Image.open(chemin_image)

# Convertir l'image en niveaux de gris
gray_image = image.convert('L')

# Augmenter le contraste de l'image
enhancer = ImageEnhance.Contrast(gray_image)
contrasted_image = enhancer.enhance(1)  # Augmenter le contraste par un facteur de 2

# Redimensionner l'image (facteur de 1.5)
width, height = contrasted_image.size
resized_image = contrasted_image.resize((int(width * 3.2), int(height * 3.2)), Image.LANCZOS)

# Binariser l'image (seuil ajusté)
binarized_image = resized_image.point(lambda x: 0 if x < 140 else 255, '1')

# Extraire le texte de l'image améliorée avec l'option psm 6
custom_config = r'--oem 3 --psm 6'
texte_bilan_enhanced_passifs = pytesseract.image_to_string(binarized_image, config=custom_config)

# texte_bilan_enhanced_passifs = re.sub(r'[\‘"|()-]', '', texte_bilan_enhanced_passifs)
# texte_bilan_enhanced_passifs = re.sub(r'08s', '085', texte_bilan_enhanced_passifs)
# texte_bilan_enhanced_passifs = re.sub(r'288', '285', texte_bilan_enhanced_passifs)
# texte_bilan_enhanced_passifs = re.sub(r'NI3Z', '', texte_bilan_enhanced_passifs)
# texte_bilan_enhanced_passifs = re.sub(r'NI2', '', texte_bilan_enhanced_passifs)
# texte_bilan_enhanced_passifs = re.sub(r'37120148', '37121148', texte_bilan_enhanced_passifs)

# texte_bilan_enhanced_passifs = re.sub(r'\brésuttat\b', 'résultat', texte_bilan_enhanced_passifs, flags=re.IGNORECASE)
# texte_bilan_enhanced_passifs = re.sub(r'\bNexercice\b', 'l exercice', texte_bilan_enhanced_passifs, flags=re.IGNORECASE)


# Fonction pour insérer le séparateur entre les deux valeurs
def insert_separator(line):
    # Expression régulière pour trouver les nombres
    numbers = re.findall(r'\d{1,3}(?:[\s.,]\d{3})*', line)
    if len(numbers) == 2:
        # Remplacer la première occurrence du deuxième nombre par '| <number>'
        line = re.sub(re.escape(numbers[1]), '| ' + numbers[1], line, 1)
    return line

def contains_keywords(text, keywords):
    """Check if the extracted text contains any of the specified keywords."""
    for keyword in keywords:
        if keyword.lower() in text.lower():
            return True
    return False
def extract_financial_data(text, keywords):
    """Extract financial data if the text contains specific keywords."""
    if contains_keywords(text, keywords):
                return "Il s'agit bien d'un bilan comptable:"
    else:
        return "The image does not appear to be a balance sheet."

# Filtrer les lignes pour ne conserver que celles contenant "Total", "total" ou "TOTAL"
lignes = texte_bilan_enhanced_passifs.split('\n')
lignes_filtrees = [insert_separator(ligne) for ligne in lignes if re.search(r'\bTotal\b', ligne, re.IGNORECASE)]

# Rejoindre les lignes filtrées en un seul texte
texte_filtre_passifs = '\n'.join(lignes_filtrees)

texte_filtre_passifs = re.sub(r'\[', '', texte_filtre_passifs)
texte_filtre_passifs = re.sub(r'\-', '', texte_filtre_passifs)
texte_filtre_passifs = re.sub(r'N12  10 085 140', 'N12   10 085 140 |', texte_filtre_passifs)
texte_filtre_passifs = re.sub(r'J', '|', texte_filtre_passifs)


keywords = ["Actifs", "Passifs", "Capitaux propres", "Résultat", "ETAT DE RESULTAT"]
extracted_value = extract_financial_data(texte_filtre_passifs, keywords)

# Afficher le texte filtré et s'il 'sagit d'un bilan ou pas
print(extracted_value)
print(texte_filtre_passifs)

Il s'agit bien d'un bilan comptable:
Total CP avant résultat de l'exercice N12   10 085 140 |  10 716 285
Total des Capitaux Propres avant affectation N13  6 368 586 |  10 085 140
Total des Passifs non Courants 1 885 454 | 1 058 590
Total des Passifs Courants 61 752 346 | 46 147 699
Total des Passifs 63 637 800 | 47 206 289
TOTAL DES CAPITAUX PROPRES & DES PASSIFS | 57 269 214 | 37 121 148


Bilan 2022 exple 1

In [5]:
!pip install pytesseract Pillow PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 53.1 MB/s eta 0:00:00


Convertir le fichier pdf en images png

In [6]:
import fitz  # PyMuPDF
from PIL import Image

# Chemin vers le fichier PDF
chemin_pdf = "/content/bilan_2022.pdf"

# Ouvrir le fichier PDF
pdf_document = fitz.open(chemin_pdf)

# Définir une résolution DPI pour le rendu des pages
dpi = 300

# Parcourir les pages du PDF et les convertir en images PNG
for page_num in range(len(pdf_document)):
    page = pdf_document[page_num]

    # Rendre la page en tant qu'image (pixmap)
    zoom = dpi / 72  # 72 DPI est la résolution de base
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, alpha=False)

    # Convertir le pixmap en une image PIL
    image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Enregistrer l'image en tant que fichier PNG
    output_path = f"page_{page_num + 1}_bilan_2022_exple1.png"
    image.save(output_path, "PNG")
    print(f"Image de la page {page_num + 1} sauvegardée: {output_path}")


Image de la page 1 sauvegardée: page_1_bilan_2022_exple1.png
Image de la page 2 sauvegardée: page_2_bilan_2022_exple1.png
Image de la page 3 sauvegardée: page_3_bilan_2022_exple1.png
Image de la page 4 sauvegardée: page_4_bilan_2022_exple1.png


Extraction des totaux des actifs pour l'exemple 1

In [7]:
from PIL import Image, ImageEnhance
import pytesseract
import re

# Charger l'image
chemin_image = "page_1_bilan_2022_exple1.png"
image = Image.open(chemin_image)

# Convertir l'image en niveaux de gris
gray_image = image.convert('L')

# Augmenter le contraste de l'image
enhancer = ImageEnhance.Contrast(gray_image)
contrasted_image = enhancer.enhance(1)  # Augmenter le contraste par un facteur de 2

# Redimensionner l'image (facteur de 1.5)
width, height = contrasted_image.size
resized_image = contrasted_image.resize((int(width * 8.3), int(height * 8.3)), Image.LANCZOS)

# Binariser l'image (seuil ajusté)
binarized_image = resized_image.point(lambda x: 0 if x < 140 else 255, '1')

# Extraire le texte de l'image améliorée avec l'option psm 6
custom_config = r'--oem 3 --psm 6'
texte_bilan_enhanced = pytesseract.image_to_string(binarized_image, config=custom_config)

# Fonction pour insérer le séparateur entre les deux valeurs
def insert_separator(line):
    # Expression régulière pour trouver les nombres
    numbers = re.findall(r'\d{1,3}(?:[\s.,]\d{3})*', line)
    if len(numbers) == 2:
        # Remplacer la première occurrence du deuxième nombre par '| <number>'
        line = re.sub(re.escape(numbers[1]), '| ' + numbers[1], line, 1)
    return line


def contains_keywords(text, keywords):
    """Check if the extracted text contains any of the specified keywords."""
    for keyword in keywords:
        if keyword.lower() in text.lower():
            return True
    return False
def extract_financial_data(text, keywords):
    """Extract financial data if the text contains specific keywords."""
    if contains_keywords(text, keywords):
                return "Il s'agit bien d'un bilan comptable:"
    else:
        return "The image does not appear to be a balance sheet."

# Filtrer les lignes pour ne conserver que celles contenant "Total", "total" ou "TOTAL"
lignes = texte_bilan_enhanced.split('\n')
lignes_filtrees = [insert_separator(ligne) for ligne in lignes if re.search(r'\bTotal\b', ligne, re.IGNORECASE)]

# Rejoindre les lignes filtrées en un seul texte
texte_bilan_2022_1_actifs = '\n'.join(lignes_filtrees)

keywords = ["Actifs", "Passifs", "Capitaux propres", "Résultat", "Resultat", "ETAT DE RESULTAT"]
extracted_value = extract_financial_data(texte_bilan_2022_1_actifs, keywords)

# Afficher le texte filtré et s'il 'sagit d'un bilan ou pas
print(extracted_value)
print(texte_bilan_2022_1_actifs)

Il s'agit bien d'un bilan comptable:
Total des actifs immobilisés 10777 848 11 096 473
Total des actifs non courants 10 777 848 | 11 096 473 :
Total des actifs courants 3 027 343 417| 9 280 :
TOTAL DES ACTIFS 13.805 191 | 15 275 753 .


Extraction des totaux des capitaux propres et des passifs pour l'exemple 1

In [8]:
from PIL import Image, ImageEnhance
import pytesseract
import re

# Charger l'image
chemin_image = "/content/page_2_bilan_2022_exple1.png"
image = Image.open(chemin_image)

# Convertir l'image en niveaux de gris
gray_image = image.convert('L')

# Augmenter le contraste de l'image
enhancer = ImageEnhance.Contrast(gray_image)
contrasted_image = enhancer.enhance(1)  # Augmenter le contraste par un facteur de 2

# Redimensionner l'image (facteur de 1.5)
width, height = contrasted_image.size
resized_image = contrasted_image.resize((int(width * 4), int(height * 4)), Image.LANCZOS)

# Binariser l'image (seuil ajusté)
binarized_image = resized_image.point(lambda x: 0 if x < 140 else 255, '1')

# Extraire le texte de l'image améliorée avec l'option psm 6
custom_config = r'--oem 3 --psm 6'
texte_bilan_enhanced = pytesseract.image_to_string(binarized_image, config=custom_config)

# Fonction pour insérer le séparateur entre les deux valeurs
def insert_separator(line):
    # Expression régulière pour trouver les nombres
    numbers = re.findall(r'\d{1,3}(?:[\s.,]\d{3})*', line)
    if len(numbers) == 2:
        # Remplacer la première occurrence du deuxième nombre par '| <number>'
        line = re.sub(re.escape(numbers[1]), '| ' + numbers[1], line, 1)
    return line


def contains_keywords(text, keywords):
    """Check if the extracted text contains any of the specified keywords."""
    for keyword in keywords:
        if keyword.lower() in text.lower():
            return True
    return False
def extract_financial_data(text, keywords):
    """Extract financial data if the text contains specific keywords."""
    if contains_keywords(text, keywords):
                return "Il s'agit bien d'un bilan comptable:"
    else:
        return "The image does not appear to be a balance sheet."

# Filtrer les lignes pour ne conserver que celles contenant "Total", "total" ou "TOTAL"
lignes = texte_bilan_enhanced.split('\n')
lignes_filtrees = [insert_separator(ligne) for ligne in lignes if re.search(r'\bTotal\b', ligne, re.IGNORECASE)]

# Rejoindre les lignes filtrées en un seul texte
texte_bilan_2022_1_passifs = '\n'.join(lignes_filtrees)
# texte_bilan_2022_1_passifs = re.sub(r'a g', '0 | 0', texte_bilan_2022_1_passifs)

keywords = ["Actifs", "Passifs", "Capitaux propres", "Résultat", "ETAT DE RESULTAT"]
extracted_value = extract_financial_data(texte_bilan_2022_1_passifs, keywords)

# Afficher le texte filtré et s'il 'sagit d'un bilan ou pas
print(extracted_value)
print(texte_bilan_2022_1_passifs)

Il s'agit bien d'un bilan comptable:
Total des capitaux propres avant resuiltat affectation 5 13 593 714 13 052 935
Total des passifs non courants 177 857 |
Total des passifs courants 211 477 | 2 104 967 :
Total des passifs _ 211 477 | 2 222 818 :
_ TOTAL DES CAPITAUX PROPRES ET DES PASSIFS 13 805 191 | 15 275 753 :


Bilan 2022 exple 2

Convertir le fichier pdf en images png

In [9]:
import fitz  # PyMuPDF
from PIL import Image

# Chemin vers le fichier PDF
chemin_pdf = "/content/bilan_2022_2.pdf"

# Ouvrir le fichier PDF
pdf_document = fitz.open(chemin_pdf)

# Définir une résolution DPI pour le rendu des pages
dpi = 300

# Parcourir les pages du PDF et les convertir en images PNG
for page_num in range(len(pdf_document)):
    page = pdf_document[page_num]

    # Rendre la page en tant qu'image (pixmap)
    zoom = dpi / 72  # 72 DPI est la résolution de base
    mat = fitz.Matrix(zoom, zoom)
    pix = page.get_pixmap(matrix=mat, alpha=False)

    # Convertir le pixmap en une image PIL
    image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Enregistrer l'image en tant que fichier PNG
    output_path = f"page_{page_num + 1}_bilan_2022_exple2.png"
    image.save(output_path, "PNG")
    print(f"Image de la page {page_num + 1} sauvegardée: {output_path}")


Image de la page 1 sauvegardée: page_1_bilan_2022_exple2.png
Image de la page 2 sauvegardée: page_2_bilan_2022_exple2.png
Image de la page 3 sauvegardée: page_3_bilan_2022_exple2.png
Image de la page 4 sauvegardée: page_4_bilan_2022_exple2.png
Image de la page 5 sauvegardée: page_5_bilan_2022_exple2.png


Extraction des totaux des actifs pour l'exemple 2

In [13]:
from PIL import Image, ImageEnhance
import pytesseract
import re

# Charger l'image
chemin_image = "/content/page_2_bilan_2022_exple2.png"
image = Image.open(chemin_image)

# Convertir l'image en niveaux de gris
gray_image = image.convert('L')

# Augmenter le contraste de l'image
enhancer = ImageEnhance.Contrast(gray_image)
contrasted_image = enhancer.enhance(2)  # Augmenter le contraste par un facteur de 2

# Redimensionner l'image (facteur de 1.5)
width, height = contrasted_image.size
resized_image = contrasted_image.resize((int(width * 3), int(height * 3)), Image.LANCZOS)

# Binariser l'image (seuil ajusté)
binarized_image = resized_image.point(lambda x: 0 if x < 140 else 255, '1')

# Extraire le texte de l'image améliorée avec l'option psm 6
custom_config = r'--oem 3 --psm 6'
texte_bilan_enhanced = pytesseract.image_to_string(binarized_image, config=custom_config)

# Fonction pour insérer le séparateur entre les deux valeurs
def insert_separator(line):
    # Expression régulière pour trouver les nombres
    numbers = re.findall(r'\d{1,3}(?:[\s.,]\d{3})*', line)
    if len(numbers) == 2:
        # Remplacer la première occurrence du deuxième nombre par '| <number>'
        line = re.sub(re.escape(numbers[1]), '| ' + numbers[1], line, 1)
    return line


def contains_keywords(text, keywords):
    """Check if the extracted text contains any of the specified keywords."""
    for keyword in keywords:
        if keyword.lower() in text.lower():
            return True
    return False
def extract_financial_data(text, keywords):
    """Extract financial data if the text contains specific keywords."""
    if contains_keywords(text, keywords):
                return "Il s'agit bien d'un bilan comptable:"
    else:
        return "The image does not appear to be a balance sheet."

# Filtrer les lignes pour ne conserver que celles contenant "Total", "total" ou "TOTAL"
lignes = texte_bilan_enhanced.split('\n')
lignes_filtrees = [insert_separator(ligne) for ligne in lignes if re.search(r'\bTotal\b', ligne, re.IGNORECASE)]

# Rejoindre les lignes filtrées en un seul texte
texte_bilan_2022_2_actifs = '\n'.join(lignes_filtrees)
texte_bilan_2022_2_actifs = re.sub(r'\~', '', texte_bilan_2022_2_actifs)
texte_bilan_2022_2_actifs = re.sub(r'\-', '', texte_bilan_2022_2_actifs)
texte_bilan_2022_2_actifs = re.sub(r'\(', '', texte_bilan_2022_2_actifs)
texte_bilan_2022_2_actifs = re.sub(r'\_', '', texte_bilan_2022_2_actifs)


keywords = ["Actifs", "Passifs", "Capitaux propres", "Résultat", "ETAT DE RESULTAT"]
extracted_value = extract_financial_data(texte_bilan_2022_2_actifs, keywords)

# Afficher le texte filtré et s'il 'sagit d'un bilan ou pas
print(extracted_value)
print(texte_bilan_2022_2_actifs)

Il s'agit bien d'un bilan comptable:
 Total des actifs immohilisés 13 411 516 | 13 694 454
Total des actifs non courants 13 418 313 | 13 705 529
 Total des actifs courants 3 305 096 | 4 094 239
 Total des actifs 16 723409 17799 768


Extraction des totaux des capitaux propres et des passifs pour l'exemple 2

In [12]:
from PIL import Image, ImageEnhance
import pytesseract
import re

# Charger l'image
chemin_image = "/content/page_3_bilan_2022_exple2.png"
image = Image.open(chemin_image)

# Convertir l'image en niveaux de gris
gray_image = image.convert('L')

# Augmenter le contraste de l'image
enhancer = ImageEnhance.Contrast(gray_image)
contrasted_image = enhancer.enhance(2)  # Augmenter le contraste par un facteur de 2

# Redimensionner l'image (facteur de 1.5)
width, height = contrasted_image.size
resized_image = contrasted_image.resize((int(width * 3), int(height * 3)), Image.LANCZOS)

# Binariser l'image (seuil ajusté)
binarized_image = resized_image.point(lambda x: 0 if x < 140 else 255, '1')

# Extraire le texte de l'image améliorée avec l'option psm 6
custom_config = r'--oem 3 --psm 6'
texte_bilan_enhanced = pytesseract.image_to_string(binarized_image, config=custom_config)

# Fonction pour insérer le séparateur entre les deux valeurs
def insert_separator(line):
    # Expression régulière pour trouver les nombres
    numbers = re.findall(r'\d{1,3}(?:[\s.,]\d{3})*', line)
    if len(numbers) == 2:
        # Remplacer la première occurrence du deuxième nombre par '| <number>'
        line = re.sub(re.escape(numbers[1]), '| ' + numbers[1], line, 1)
    return line


def contains_keywords(text, keywords):
    """Check if the extracted text contains any of the specified keywords."""
    for keyword in keywords:
        if keyword.lower() in text.lower():
            return True
    return False
def extract_financial_data(text, keywords):
    """Extract financial data if the text contains specific keywords."""
    if contains_keywords(text, keywords):
                return "Il s'agit bien d'un bilan comptable:"
    else:
        return "The image does not appear to be a balance sheet."

# Filtrer les lignes pour ne conserver que celles contenant "Total", "total" ou "TOTAL"
lignes = texte_bilan_enhanced.split('\n')
lignes_filtrees = [insert_separator(ligne) for ligne in lignes if re.search(r'\bTotal\b', ligne, re.IGNORECASE)]

# Rejoindre les lignes filtrées en un seul texte
texte_bilan_2022_2_passifs = '\n'.join(lignes_filtrees)
texte_bilan_2022_2_passifs = re.sub(r'\:', '', texte_bilan_2022_2_passifs)
texte_bilan_2022_2_passifs = re.sub(r'7 ', '', texte_bilan_2022_2_passifs)
# texte_bilan_2022_2_passifs = re.sub(r'| Total', 'Total', texte_bilan_2022_2_passifs)
texte_bilan_2022_2_passifs = re.sub(r'a g', '0 | 0', texte_bilan_2022_2_passifs)



keywords = ["Actifs", "Passifs", "Capitaux propres", "Résultat", "ETAT DE RESULTAT"]
extracted_value = extract_financial_data(texte_bilan_2022_2_passifs, keywords)

# Afficher le texte filtré et s'il 'sagit d'un bilan ou pas
print(extracted_value)
print(texte_bilan_2022_2_passifs)

Il s'agit bien d'un bilan comptable:
Total des capitaux propres avant affectation 16 360 904 | 15 716 941
Total des passifs non courants 0 | 0
| Total des passifs courants 362 505 | 2 082 827
Total des passifs 362 505 | 2 082 827
 Total des capitaux propres et passifs 16 723 409 | 1799 768
